In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set() 
from sklearn.datasets.samples_generator import make_blobs
%matplotlib inline

d:\desktop\github\m2-project\twittervenv\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
df = pd.read_csv('all.csv', header=0)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [28]:
vectorizer = TfidfVectorizer(max_df=0.5, max_features=len(df.columns), min_df=2, stop_words='english', use_idf=True)

In [29]:
X = vectorizer.fit_transform(df.text)

In [30]:
X

<51683x16 sparse matrix of type '<class 'numpy.float64'>'
	with 143917 stored elements in Compressed Sparse Row format>

In [35]:
K = 4
km = KMeans(n_clusters=K, init='k-means++', max_iter=100, n_init=1, verbose=True)

In [36]:
km.fit(X)

Initialization complete
Iteration 0, inertia 40506.47218120042
Iteration 1, inertia 29699.86884733076
Iteration 2, inertia 27972.928990911252
Iteration 3, inertia 27574.022614435755
Iteration 4, inertia 27532.68160015229
Iteration 5, inertia 27521.727876391906
Iteration 6, inertia 27518.086504585666
Iteration 7, inertia 27515.74498053862
Iteration 8, inertia 27514.429192053958
Iteration 9, inertia 27514.180944157004
Iteration 10, inertia 27514.053118978863
Iteration 11, inertia 27513.768295335736
Converged at iteration 11: center shift 4.392589089710315e-06 within tolerance 4.905331742909397e-06.


KMeans(max_iter=100, n_clusters=4, n_init=1, verbose=True)

In [37]:
set(km.labels_)

{0, 1, 2, 3}

In [38]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]

terms = vectorizer.get_feature_names()
for i in range(K):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()

Cluster 0: كورونا لبنان في كورونا_لبنان خليك_بالبيت على من com https twitter
Cluster 1: medical lebanon http https com healthcare twitter لبنان كورونا_لبنان في
Cluster 2: من ما في كورونا كورونا_لبنان على لبنان خليك_بالبيت twitter com
Cluster 3: كورونا_لبنان healthcare lebanon com twitter http https pic خليك_بالبيت كورونا


In [40]:
def cluster(df, verbose=False):
    
    # vectorise text data
    vectorizer = TfidfVectorizer(max_df=0.5, max_features=len(df.columns), min_df=2, stop_words='english', use_idf=True)
    X = vectorizer.fit_transform(df.text)
    
    # clustering
    K = 4
    km = KMeans(n_clusters=K, init='k-means++', max_iter=100, n_init=1, verbose=verbose)
    km.fit(X)
    
    # handle results
    order_centroids = km.cluster_centers_.argsort()[:, ::-1]
    for i in range(K):
        print("Cluster %d:" % i, end='')
        for ind in order_centroids[i, :10]:
            print(' %s' % terms[ind], end='')
        print()
    

mo7afazat = df.groupby('MOHAFAZA_AR')
for n, g in mo7afazat:
    print(f'[ {n} ]')
    cluster(g)
    print('-----------------------------------')
    

[ البقاع ]
Cluster 0: كورونا لبنان في ما على كورونا_لبنان https medical من lebanon
Cluster 1: كورونا_لبنان من في ما كورونا على لبنان medical http com
Cluster 2: pic com خليك_بالبيت twitter medical http healthcare كورونا كورونا_لبنان لبنان
Cluster 3: من healthcare كورونا في على لبنان كورونا_لبنان ما http medical
-----------------------------------
[ الجنوب ]
Cluster 0: http medical https com pic healthcare lebanon خليك_بالبيت twitter لبنان
Cluster 1: كورونا لبنان من في على ما com خليك_بالبيت كورونا_لبنان lebanon
Cluster 2: كورونا_لبنان من ما كورونا في لبنان على medical com lebanon
Cluster 3: twitter medical https lebanon com healthcare pic خليك_بالبيت لبنان كورونا_لبنان
-----------------------------------
[ الشمال ]
Cluster 0: من في على كورونا_لبنان كورونا ما لبنان خليك_بالبيت com https
Cluster 1: pic كورونا_لبنان twitter medical http lebanon https healthcare com كورونا
Cluster 2: com خليك_بالبيت https كورونا في لبنان من http كورونا_لبنان ما
Cluster 3: كورونا لبنان ما في كورونا_لبنان عل

In [47]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD

def cluster_with_lsa(df, verbose=False):
    
    features = len(df.columns)
    
    # vectorise text data
    vectorizer = TfidfVectorizer(max_df=0.5, max_features=features, min_df=2, stop_words='english', use_idf=True)
    X = vectorizer.fit_transform(df.text)
    
    # LSA
    svd = TruncatedSVD(features - 1)
    normalizer = Normalizer(copy=False)
    lsa = make_pipeline(svd, normalizer)
    X = lsa.fit_transform(X)
    
    # clustering
    K = 4
    km = KMeans(n_clusters=K, init='k-means++', max_iter=100, n_init=1, verbose=verbose)
    km.fit(X)
    
    # handle results
    order_centroids = km.cluster_centers_.argsort()[:, ::-1]
    for i in range(K):
        print("Cluster %d:" % i, end='')
        for ind in order_centroids[i, :10]:
            print(' %s' % terms[ind], end='')
        print()
        
cluster_with_lsa(df)

Cluster 0: com https healthcare twitter لبنان كورونا lebanon خليك_بالبيت في كورونا_لبنان
Cluster 1: healthcare com medical pic خليك_بالبيت في كورونا على كورونا_لبنان ما
Cluster 2: http healthcare com https في twitter كورونا_لبنان lebanon لبنان ما
Cluster 3: com lebanon لبنان http على medical في ما خليك_بالبيت pic


In [48]:
mo7afazat = df.groupby('MOHAFAZA_AR')
for n, g in mo7afazat:
    print(f'[ {n} ]')
    cluster_with_lsa(g)
    print('-----------------------------------')

[ البقاع ]
Cluster 0: com twitter على lebanon في https medical pic كورونا كورونا_لبنان
Cluster 1: com http healthcare كورونا_لبنان كورونا pic لبنان twitter ما lebanon
Cluster 2: healthcare https com medical في http خليك_بالبيت pic على ما
Cluster 3: com https خليك_بالبيت medical كورونا في لبنان ما كورونا_لبنان lebanon
-----------------------------------
[ الجنوب ]
Cluster 0: healthcare https com lebanon في pic على ما خليك_بالبيت لبنان
Cluster 1: http com لبنان على https كورونا_لبنان في healthcare twitter medical
Cluster 2: healthcare lebanon com كورونا في twitter ما لبنان medical http
Cluster 3: com medical pic على lebanon كورونا_لبنان في twitter خليك_بالبيت ما
-----------------------------------
[ الشمال ]
Cluster 0: com healthcare خليك_بالبيت pic لبنان lebanon twitter كورونا_لبنان medical في
Cluster 1: com http https medical كورونا_لبنان في على لبنان ما خليك_بالبيت
Cluster 2: https healthcare com lebanon على http medical كورونا pic ما
Cluster 3: http healthcare com في medical على twit